In [1]:
import numpy as np
import tensorflow as tf
import cv2
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
print(tf.__version__)

In [ ]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability = None
)

print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))

In [ ]:
#Image Preprocessor class

class ImagePreprocessor:
    def __init__(self, root_dir, desired_size=(128, 128)):
        self.root_dir = root_dir
        self.desired_size = desired_size
        ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator

        self.datagen = ImageDataGenerator(
            rotation_range=20,
            zoom_range=0.15,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.15,
            horizontal_flip=True,
            fill_mode="nearest"
        )
        
    def _load_image(self, path):
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
        image = cv2.resize(image, self.desired_size)
        return image
    
    def process_images(self):
        images = []
        labels = []

        for label in os.listdir(self.root_dir):
            class_path = os.path.join(self.root_dir, label)
            if os.path.isdir(class_path):
                for filename in os.listdir(class_path):
                    filepath = os.path.join(class_path, filename)
                    if filepath.lower().endswith(('.jpg', '.jpeg')):
                        image = self._load_image(filepath)
                        images.append(image)
                        labels.append(label)
        
        # Convert images and labels to arrays
        images = np.array(images) / 255.0
        labels = np.array(labels)
        print("Unique Labels: ", np.unique(labels))

        # One-hot encode labels
        lb = LabelBinarizer()
        labels = lb.fit_transform(labels)
        return images, labels

    def get_train_val_test(self, test_size=0.2, val_size=0.1):
        images, labels = self.process_images()
        trainX, testX, trainY, testY = train_test_split(images, labels, test_size=test_size)
        trainX, valX, trainY, valY = train_test_split(trainX, trainY, test_size=val_size / (1 - test_size))
        
        return (trainX, trainY), (valX, valY), (testX, testY)

    def save_processed_images(self, save_dir):

        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        images, labels = self.process_images()

        #Save images as file name

        for idx, (image, label) in enumerate(zip(images, labels)):
            label_str = str(np.argmax(label))
            filename = f"{label_str}_{idx}.jpg"
            filepath = os.path.join(save_dir, filename)

            # Convert the image back to the range [0, 255]
            image_to_save = (image * 255).astype(np.uint8)

            cv2.imwrite(filepath, cv2.cvtColor(image_to_save, cv2.COLOR_RGB2BGR))


In [ ]:
#Load MobileNetV2 without top layer

base_model = MobileNetV2(weights='imagenet', include_top=False)

In [ ]:
#Average Pooling Layer

x = base_model.output
x = GlobalAveragePooling2D()(x)

#fully-connected layer 

x = Dense(1024, activation = 'relu')(x)

# softmax layer for classification

num_classes = len(os.listdir("C:\\Users\\bcurl\\Desktop\\AnimalDetect\\data\\raw-img")) 

predictions = Dense(num_classes, activation='softmax')(x)

#trained models

model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
#Freeze pre trained layers

for layer in base_model.layers:
    layer.trainable = False
    

In [ ]:
#Compile the model 

model.compile(optimizer = 'adam', loss='categorical_crossentropy',metrics =['accuracy'])

In [ ]:
#process images 

preprocessor = ImagePreprocessor("C:\\Users\\bcurl\\Desktop\\AnimalDetect\\data\\raw-img")
(trainX, trainY), (valX, valY), (testX, testY) = preprocessor.get_train_val_test()

In [ ]:
#Train the model with the new layers for some epochs

model.fit(trainX, trainY, validation_data=(valX, valY), epochs=10, batch_size=32)